# Window Functions com DuckDB

* [Window Functions](https://duckdb.org/docs/stable/sql/functions/window_functions)

In [ ]:
import duckdb

In [ ]:
conn = duckdb.connect()

In [ ]:
conn.execute("""
    CREATE OR REPLACE TABLE vendas (
        id INTEGER,
        vendedor TEXT,
        regiao TEXT,
        valor NUMERIC,
        data DATE
    );
""")

In [ ]:
dados = [
    (1, 'Ana',    'Sul',   1000, '2024-01-01'),
    (2, 'Bruno',  'Sul',   1500, '2024-01-02'),
    (3, 'Carla',  'Norte', 1200, '2024-01-03'),
    (4, 'Ana',    'Sul',   900,  '2024-01-04'),
    (5, 'Bruno',  'Sul',   2000, '2024-01-05'),
    (6, 'Carla',  'Norte', 1100, '2024-01-06'),
    (7, 'Ana',    'Sul',   1500, '2024-01-07')
]

* [Prepared Statements](https://duckdb.org/docs/stable/clients/python/dbapi#prepared-statements)

In [ ]:
conn.executemany(
    "INSERT INTO vendas VALUES (?, ?, ?, ?, ?);",
    dados
)

* [Aggregate Window Functions](https://duckdb.org/docs/stable/sql/functions/window_functions#aggregate-window-functions)

In [ ]:
display(conn.execute("""
    SELECT
        vendedor,
        data,
        valor,
        SUM(valor) OVER (PARTITION BY vendedor ORDER BY data) AS soma_acumulada
    FROM vendas;
""").df())

* [RANK](https://duckdb.org/docs/stable/sql/functions/window_functions#rankorder-by-ordering)

In [ ]:
display(conn.execute("""
    SELECT
        vendedor,
        valor,
        regiao,
        RANK() OVER (PARTITION BY regiao ORDER BY valor DESC) AS ranking
    FROM vendas;
""").df())

* [LAG](https://duckdb.org/docs/stable/sql/functions/window_functions#lagexpr-offset-default-order-by-ordering-ignore-nulls)

In [ ]:
display(conn.execute("""
    SELECT
        vendedor,
        data,
        valor,
        LAG(valor) OVER (PARTITION BY vendedor ORDER BY data) AS valor_anterior
    FROM vendas;
""").df())

* [NTILE](https://duckdb.org/docs/stable/sql/functions/window_functions#ntilenum_buckets-order-by-ordering)

In [ ]:
display(conn.execute("""
    SELECT
        vendedor,
        valor,
        NTILE(4) OVER (ORDER BY valor DESC) AS quartil
    FROM vendas;
""").df())

In [ ]:
conn.close()